This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5rc2) and depends on Magma (tested using version 2.26-9).

In this notebook, we read the data files of Dragutinović tabulating curves of genus 5 and produce a new output file `genus5byweilpoly.txt`. The latter is a dictionary of curves of genus 5 keyed by their Weil polynomial (in the variable `T`). 
- Each hyperelliptic curve is represented as a pair $(f,h)$ as in Sage, in the variable `x`.
- Each trigonal curve is represented as a singular plane curve in the variables `x0, x1, x2`.
- Each complete intersection is represented as a triple of generators in the variables `x0, x1, x2, x3, x4`.

Allow 5 minutes for completion.

In [1]:
load("../Shared/weil_poly_utils.sage")
import itertools
from collections import defaultdict

Read and process data on hyperelliptic curves, computing zeta functions using Sage.

In [2]:
P.<T> = QQ[]
hyperelliptic_curves_by_poly = defaultdict(list)
trigonal_curves_by_poly = defaultdict(list)
ci_curves_by_poly = defaultdict(list)

In [3]:
P1.<x> = GF(2)[]
with open("HyperellipticCurvesData.txt", "r") as f:
    s = f.read()
    l = sage_eval(s, locals={'x':P1.gen()})
for (h, g) in l:
    C = HyperellipticCurve(g,h)
    assert C.genus() == 5
    u = C.frobenius_polynomial()(T)
    hyperelliptic_curves_by_poly[u].append((h,g))

Read and process data on trigonal curves, represented as plane quintics with a single double point (not necessarily a node), computing zeta functions using Magma.

In [4]:
P3.<x0,x1,x2> = GF(2)[]
with open("TrigonalCurvesWithAutomorphisms.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
monos5 = [prod(t) for t in itertools.combinations_with_replacement(P3.gens(), 5)]
proj = magma.ProjectiveSpace(P3)
for (v, _) in l:
    gen = sum(v[i]*monos5[i] for i in range(len(monos5)))
    X = proj.Scheme([gen])
    assert X.Dimension() == 1
    C = X.Curve()
    assert C.Genus() == 5
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 6))
    u = weil_poly_from_point_count(ct, 5)(T)
    trigonal_curves_by_poly[u].append(gen)

Read and process data on complete intersections of type $(2,2,2)$ in $\mathbb{P}^4$, using precomputed zeta functions. These will be verified elsewhere.

In [5]:
P5.<x0,x1,x2,x3,x4> = GF(2)[]
with open("Complete_Intersections.txt", "r") as f:
    s = f.read()
    l1 = sage_eval(s)
with open("Pts_Count_Complete_Intersections.txt", "r") as f:
    s = f.read()
    l2 = sage_eval(s)
monos2 = [prod(t) for t in itertools.combinations_with_replacement(P5.gens(), 2)]
for i in range(len(l1)):
    gens = [sum(v[j]*monos2[j] for j in range(len(monos2))) for v in l1[i]]
    u = weil_poly_from_point_count(l2[i], 5)(T)
    ci_curves_by_poly[u].append(gens)

Generate the output file.

In [6]:
d = {u: [hyperelliptic_curves_by_poly[u], trigonal_curves_by_poly[u], ci_curves_by_poly[u]] \
     for u in set(hyperelliptic_curves_by_poly) | set(trigonal_curves_by_poly) | set(ci_curves_by_poly)}
with open("genus5byweilpoly.txt", "w") as f:
    f.write(str(d))

In [8]:
l = [[0,0,9,12,50], [0,0,12,16,60], [0,4,0,16,20]]
for t in l:
    u = weil_poly_from_point_count(ct, 5)(T)
    s = label_from_weil_poly(u)
    print(s, d[u])

5.2.b_b_ab_a_a [[], [x0^4*x1 + x0*x1^4 + x1^5 + x0*x1^3*x2 + x0^2*x1*x2^2 + x0*x1^2*x2^2 + x0^2*x2^3 + x0*x1*x2^3, x0^4*x1 + x0^3*x1^2 + x0^2*x1^3 + x0*x1^4 + x1^5 + x0^3*x1*x2 + x0^2*x2^3 + x0*x1*x2^3], [[x1^2 + x1*x2 + x2^2 + x0*x3 + x2*x3, x0^2 + x1^2 + x2^2 + x0*x3 + x0*x4 + x1*x4 + x3*x4, x0^2 + x0*x1 + x1^2 + x0*x2 + x2^2 + x2*x3 + x3^2 + x2*x4 + x3*x4 + x4^2]]]
5.2.b_b_ab_a_a [[], [x0^4*x1 + x0*x1^4 + x1^5 + x0*x1^3*x2 + x0^2*x1*x2^2 + x0*x1^2*x2^2 + x0^2*x2^3 + x0*x1*x2^3, x0^4*x1 + x0^3*x1^2 + x0^2*x1^3 + x0*x1^4 + x1^5 + x0^3*x1*x2 + x0^2*x2^3 + x0*x1*x2^3], [[x1^2 + x1*x2 + x2^2 + x0*x3 + x2*x3, x0^2 + x1^2 + x2^2 + x0*x3 + x0*x4 + x1*x4 + x3*x4, x0^2 + x0*x1 + x1^2 + x0*x2 + x2^2 + x2*x3 + x3^2 + x2*x4 + x3*x4 + x4^2]]]
5.2.b_b_ab_a_a [[], [x0^4*x1 + x0*x1^4 + x1^5 + x0*x1^3*x2 + x0^2*x1*x2^2 + x0*x1^2*x2^2 + x0^2*x2^3 + x0*x1*x2^3, x0^4*x1 + x0^3*x1^2 + x0^2*x1^3 + x0*x1^4 + x1^5 + x0^3*x1*x2 + x0^2*x2^3 + x0*x1*x2^3], [[x1^2 + x1*x2 + x2^2 + x0*x3 + x2*x3, x0^2 + x1^2 + x